In [87]:
import pandas as pd
import numpy as np
import os


def bankyear(df_):
    df = df_.copy()
    years = [2016, 2017, 2018, 2019]
    df['bankrupt-year'] = 0
    for firm in df.index:
        unchanged = True
        for year in years:
            if df.loc[firm, 'evento'+str(year)]==1 and unchanged==True:
                df.loc[firm,'bankrupt-year']= year
                unchanged = False
    return df['bankrupt-year']



def new_firms(current_df, next_df, key):
    firm_first =  set(current_df[key].unique()) -  set(next_df[key].unique())
    firm_second =  set(next_df[key].unique()) -  set(current_df[key].unique())
    return pd.concat([current_df[current_df[key].isin(firm_first)] , next_df[next_df[key].isin(firm_second)]])


In [88]:
os.chdir(r"C:\Users\iván\Desktop\Thesis\Thesis-bankrupt\Datasets")

In [89]:
# Loading datasets
dfs = {}
years = [2016,2017,2018,2019]
for year in years:
      dfs['df'+str(year)]  = pd.read_excel('Carátula-Plenas-'+str(year)+'.xlsx')

In [90]:
df = pd.read_excel("Carátula-Plenas-2019.xlsx")

In [91]:
for year in years:
    print('#'*10)
    print(dfs['df'+str(year)].shape , year)
    dfs['df'+str(year)]  = dfs['df'+str(year)][dfs['df'+str(year)]['Estado actual']!='EN ETAPA PREOPERATIVA']
    print(dfs['df'+str(year)].shape , year)
    dfs['df'+str(year)]['Fecha'] = pd.to_datetime( dfs['df'+str(year)][ 'Fecha de Corte'], format= "%Y-%m-%d")
    dfs['df'+str(year)]  = dfs['df'+str(year)][dfs['df'+str(year)]['Fecha'].dt.month==12] # there is duplicates given present the information two or more times..
    print(dfs['df'+str(year)].shape , year)
    dfs['df'+str(year)] =  dfs['df'+str(year)][-pd.isnull(dfs['df'+str(year)]['NIT'])]
    print(dfs['df'+str(year)].shape , year)
    dfs['df'+str(year)]['evento'+str(year)]  = np.where(dfs['df'+str(year)]['Estado actual']!='ACTIVA',1 ,0)
    dfs['df'+str(year)] = dfs['df'+str(year)][['evento'+str(year), 'NIT' ,'Clasificación Industrial Internacional Uniforme Versión 4 A.C']]
    print(dfs['df'+str(year)]['NIT'].duplicated().sum())
    print(dfs['df'+str(year)].shape , year)

##########
(2515, 32) 2016
(2492, 32) 2016
(2482, 33) 2016
(2482, 33) 2016
0
(2482, 3) 2016
##########
(2433, 33) 2017
(2411, 33) 2017
(2399, 34) 2017
(2399, 34) 2017
0
(2399, 3) 2017
##########
(2466, 35) 2018
(2447, 35) 2018
(2438, 36) 2018
(2438, 36) 2018
0
(2438, 3) 2018
##########
(2666, 37) 2019
(2647, 37) 2019
(2637, 38) 2019
(2636, 38) 2019
0
(2636, 3) 2019


In [92]:
for year in range(2016,2020):
    print(dfs['df'+str(year)]['evento'+str(year)].value_counts())

evento2016
0    2435
1      47
Name: count, dtype: int64
evento2017
0    2335
1      64
Name: count, dtype: int64
evento2018
0    2365
1      73
Name: count, dtype: int64
evento2019
0    2558
1      78
Name: count, dtype: int64


In [93]:
full = dfs['df2016'].copy()
for year in range(2017,2020):
    clients= new_firms(full,dfs['df'+str(year)], 'NIT')
    full = full.merge(dfs['df'+str(year)], on='NIT', how='inner', suffixes=(None, 'to_remove'))
    duplicated = [x for x in full.columns if 'to_remove' in x]
    full.drop(columns=duplicated, inplace=True)
    full = pd.concat([full, clients])

In [94]:
full

,evento2016,NIT,Clasificación Industrial Internacional Uniforme Versión 4 A.C,evento2017,evento2018,evento2019
0,0.0,900146783.0,M7110 - Actividades de arquitectura e ingenier...,0.0,0.0,0.0
1,0.0,830126832.0,G4642 - Comercio al por mayor de prendas de ve...,0.0,0.0,0.0
2,0.0,830127076.0,B0722 - Extracción de oro y otros metales prec...,0.0,0.0,0.0
3,0.0,830127647.0,G4664 - Comercio al por mayor de productos quí...,0.0,0.0,0.0
4,0.0,830127761.0,A0125 - Cultivo de flor de corte,0.0,0.0,0.0
...,...,...,...,...,...,...
2661,NaN,901321213.0,K6499 - Otras actividades de servicio financie...,NaN,NaN,0.0
2662,NaN,901323081.0,J6201 - Actividades de desarrollo de sistemas ...,NaN,NaN,0.0
2663,NaN,901331867.0,K6613 - Otras actividades relacionadas con el ...,NaN,NaN,0.0
2664,NaN,830143051.0,M7020 - Actividades de consultaría de gestión,NaN,NaN,0.0


In [95]:
full[['evento2016', 'evento2017', 'evento2018', 'evento2019']].sum(axis=1, min_count=1).value_counts(dropna=False)

0.0    3121
1.0      33
4.0      27
3.0      25
2.0      23
Name: count, dtype: int64

In [96]:
full = full[-full['NIT'].duplicated()].reset_index(drop=True)
full['time-event'] =bankyear(full)
full['time-event'].value_counts(dropna=False)

time-event
0       3121
2016      47
2017      23
2019      20
2018      18
Name: count, dtype: int64

In [97]:
full['event'] = (full['time-event']!=0).astype('int')

In [98]:
full['event'].value_counts()

event
0    3121
1     108
Name: count, dtype: int64

In [99]:
full['event'].value_counts()

event
0    3121
1     108
Name: count, dtype: int64

In [100]:
full[-(full['evento2016']==1)]['event'].value_counts()

event
0    3121
1      61
Name: count, dtype: int64

In [101]:
full

,evento2016,NIT,Clasificación Industrial Internacional Uniforme Versión 4 A.C,evento2017,evento2018,evento2019,time-event,event
0,0.0,900146783.0,M7110 - Actividades de arquitectura e ingenier...,0.0,0.0,0.0,0,0
1,0.0,830126832.0,G4642 - Comercio al por mayor de prendas de ve...,0.0,0.0,0.0,0,0
2,0.0,830127076.0,B0722 - Extracción de oro y otros metales prec...,0.0,0.0,0.0,0,0
3,0.0,830127647.0,G4664 - Comercio al por mayor de productos quí...,0.0,0.0,0.0,0,0
4,0.0,830127761.0,A0125 - Cultivo de flor de corte,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...
3224,NaN,901321213.0,K6499 - Otras actividades de servicio financie...,NaN,NaN,0.0,0,0
3225,NaN,901323081.0,J6201 - Actividades de desarrollo de sistemas ...,NaN,NaN,0.0,0,0
3226,NaN,901331867.0,K6613 - Otras actividades relacionadas con el ...,NaN,NaN,0.0,0,0
3227,NaN,830143051.0,M7020 - Actividades de consultaría de gestión,NaN,NaN,0.0,0,0
